In [1]:
import numpy as np
import sys; sys.path.append('/mnt/pns/home/sotomayorb/git/spyke-tools-dev/')

In [2]:
%load_ext autoreload
from spyketools.proc.distances.spikeship import spikeship_distance_single
from spyketools.proc.distances.SPIKE_distance import SPIKE_distance_single
from spyketools.proc.distances.spike_metrics import pairwise_spike_distance
from spyketools.proc.distances.SPIKE_distance import SPIKE_distance_single
from spyketools.proc.distances.RISPIKE_distance import RISPIKE_distance_single
from spyketools.proc.distances.victor_purpura_distance import VP_distance_single
from spyketools.proc.distances.ISI_distance import ISI_distance_single
%autoreload 2

/home/sotomayorb/anaconda3/envs/allensdk_SS/lib/python3.6/site-packages/numba/core/decorators.py:232: RuntimeWarning: nopython is set for njit and is ignored
  warnings.warn('nopython is set for njit and is ignored', RuntimeWarning)


### 1. Computation of distances between two spike trains

In [3]:
# creating random data
firing_rate = 20 # num. of spikes per sec
spike_train_i = np.random.randint(1000, size=firing_rate)/1000.
spike_train_j = np.random.randint(1000, size=firing_rate)/1000.

In [4]:
%%time
spikeship_distance_single(t1=spike_train_i, t2=spike_train_j)

CPU times: user 8.62 ms, sys: 1.75 ms, total: 10.4 ms
Wall time: 9.86 ms


0.24915

In [5]:
%%time
SPIKE_distance_single(t1=spike_train_i, t2=spike_train_j, t_start=0, t_end=1)

CPU times: user 3.43 s, sys: 18 ms, total: 3.44 s
Wall time: 3.45 s


68914.07819943024

In [6]:
%%time
RISPIKE_distance_single(t1=spike_train_i, t2=spike_train_j, t_start=0, t_end=1)

CPU times: user 1.81 s, sys: 7.29 ms, total: 1.82 s
Wall time: 1.82 s


106.68410755071635

In [7]:
%%time
VP_distance_single(t1=spike_train_i, t2=spike_train_j, cost=firing_rate)

CPU times: user 1.5 s, sys: 130 µs, total: 1.5 s
Wall time: 1.51 s


14.620000000000001

In [8]:
%%time
ISI_distance_single(t1=spike_train_i, t2=spike_train_j, t_start=0, t_end=0.1)

CPU times: user 917 ms, sys: 2.06 ms, total: 919 ms
Wall time: 921 ms


0.19437331398767854

### 2. Execution time difference between Python+NumPy and Numba implementations

For this example, we will consider SPIKE distance. To run any distance from SpykeTools, we set the value of the parameter `mode = 'py'`.

In [9]:
from time import time

In [10]:
exec_time = time()
SPIKE_distance_single(t1=spike_train_i, t2=spike_train_j, t_start=0, t_end=1, mode='py')
exec_time = time() - exec_time
print ("Exec. time = %.6f" % exec_time)

Exec. time = 0.000823


In [11]:
exec_time = time()
SPIKE_distance_single(t1=spike_train_i, t2=spike_train_j, t_start=0, t_end=1, mode='njit')
exec_time = time() - exec_time
print ("Exec. time = %.6f" % exec_time)

Exec. time = 0.000077


In [12]:
n_repetitions = 1000
total_exec_time_py   = 0.
total_exec_time_njit = 0.
for i in range(n_repetitions):
    spike_train_i = np.random.randint(1000, size=firing_rate)/1000.
    spike_train_j = np.random.randint(1000, size=firing_rate)/1000.
    
    exec_time = time()
    SPIKE_distance_single(t1=spike_train_i, t2=spike_train_j, t_start=0, t_end=1, mode='py')
    exec_time = time() - exec_time
    total_exec_time_py += exec_time
    
    exec_time = time()
    SPIKE_distance_single(t1=spike_train_i, t2=spike_train_j, t_start=0, t_end=1, mode='njit')
    exec_time = time() - exec_time
    total_exec_time_njit += exec_time
    
total_exec_time_py   /= n_repetitions
total_exec_time_njit /= n_repetitions

In [13]:
print ("Exec. time = %.8f sec (python + numpy)  " % total_exec_time_py)
print ("Exec. time = %.8f sec (   compiled   )" % total_exec_time_njit)
print ("%.3f times faster! (%i repetitions)" % (total_exec_time_py/total_exec_time_njit, n_repetitions))

Exec. time = 0.00056659 sec (python + numpy)  
Exec. time = 0.00000471 sec (   compiled   )
120.181 times faster! (1000 repetitions)


Note: By default, `mode = 'njit'` which correspond to "no-python" mode of Numba's **J**ust-**I**n-**T**ime compilation).

---